In [1]:
%pip install langchain langchain-openai langchain-community pypdf faiss-cpu openai tiktoken "giskard[llm]" -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import nest_asyncio
import warnings
import os
from dotenv import load_dotenv

nest_asyncio.apply()

warnings.filterwarnings('ignore')
_ = load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LLAMAPARSE_API_KEY = os.getenv('LLAMACLOUD_API_KEY')

In [2]:
from langchain import FAISS, PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

policy_doc_location = "docs/pb116349-business-health-select-handbook-1024-pdfa.pdf"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)
loader = PyPDFLoader(policy_doc_location)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

Loaded 48 documents


In [3]:
db = FAISS.from_documents(loader.load_and_split(text_splitter), OpenAIEmbeddings())

# Prepare QA chain
PROMPT_TEMPLATE = """You are the insurence policy Assistant, a helpful AI assistant made by Giskard.
Your task is to answer common questions on insurence policies.
You will be given a question and relevant excerpts from the insurence policy documents.
Please provide short and clear answers based on the provided context. Be polite and helpful.

Context:
{context}

Question:
{question}

Your answer:
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["question", "context"])
policy_qa_chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever(), prompt=prompt)

# Test that everything works
policy_qa_chain.invoke({"query": "How much can I claim for optical expenses?"})

{'query': 'How much can I claim for optical expenses?',
 'result': 'You can claim 80% of the cost of prescribed glasses and contact lenses, up to £200 a year. Additionally, you can claim up to £25 a year for an eye test. If you have an excess, you do not have to pay it when claiming for these optical expenses.'}

In [4]:
import giskard
import pandas as pd

def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function. The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [policy_qa_chain.invoke({"query": question}) for question in df["question"]]

giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Insurence policy document question answering",
    description="This model answers any question about the insurence policy document",
    feature_names=["question"],
)

2024-11-06 10:34:54,214 pid:2768 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [5]:
# Optional: let’s test that the wrapped model works
examples = [
    "Whats the cashback amount for optical expenses?",
    "Whats the cashback amount for dental expenses?",
]
giskard_dataset = giskard.Dataset(pd.DataFrame({"question": examples}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)

2024-11-06 10:35:02,039 pid:2768 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


2024-11-06 10:35:02,096 pid:2768 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-11-06 10:35:04,295 pid:2768 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (2, 1) executed in 0:00:02.254999
[{'query': 'Whats the cashback amount for optical expenses?', 'result': 'The cashback amount for optical expenses is 80% of the cost of prescribed glasses and contact lenses, up to £200 a year.'}
 {'query': 'Whats the cashback amount for dental expenses?', 'result': 'The cashback amount for dental expenses is 80% of your dentist’s fees, up to £400 a year.'}]


Scan the model for all the vulnerabilities

In [ ]:
full_report = giskard.scan(giskard_model, giskard_dataset)
display(full_report)

In [ ]:
full_report.to_html("scan_report.html")

In [ ]:
test_suite = full_report.generate_test_suite(name="Test suite generated by scan")
test_suite.run()